In [1]:
# import sys
# !{sys.executable} -m pip install --index-url https://test.pypi.org/simple/ --no-deps smartload

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import gspread
import scipy.io
import smartload.smartload as smart
import pickle
import os
from tqdm.notebook import tqdm
# from google.oauth2.credentials import Credentials

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline

from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint

In [2]:
scope = ['https://www.googleapis.com/auth/drive.file',
         "https://www.googleapis.com/auth/drive",
         "https://spreadsheets.google.com/feeds",
         'https://www.googleapis.com/auth/spreadsheets']
creds = ServiceAccountCredentials.from_json_keyfile_name("creds/creds.json", scope)
client = gspread.authorize(creds)
sheet = client.open("BehaviorOptics Experiments Records").get_worksheet(1)

data = sheet.get_all_records()

googledb = pd.DataFrame(data)
googledb = googledb[googledb['Animal ID'] != '']

assert(len(googledb.columns) == 17)
googledb.columns = ['date', 'animal', 'time_start', 'time_end', 'experimenter',
       'rig', 'stage', 'pupil', 'dropped_frames',
       'data_extraction', 'ntrials', 'nblocks',
       'water_earned', 'good_for_analyzing', 'notes',
       'imaging_power_blue', 'imaging_power_violet']

# Some data cleaning
googledb.animal = np.char.lower(np.array(googledb.animal).astype('str'))
googledb.pupil[googledb.pupil == ''] = 'FALSE'


In [3]:
# Load matlab data log
data = smart.loadmat('/Users/minhnhatle/Documents/ExternalCode/rigbox_analysis/dbmake/logdb.mat')
vals = data['logtable']['table']['data']
colnames = data['logtable']['columns']

datadict = {}
for colname, val in zip(colnames, vals):
    datadict[colname] = val

matlablog = pd.DataFrame(datadict)

In [4]:
def parse_files_for_trial_info(files):
    ntrials = []
    nblocks = []
    flags = []

    for file in tqdm(files, position=0, leave=True):
        date, sessid, animal = file.split('_')
    #     if animal != 'f01':
    #         continue
        path = os.path.join(rigboxdir, animal, date, sessid, file + '_Block.mat')
        try:
            data = smart.loadmat(path)
        except:
            ntrials.append(np.nan)
            nblocks.append(np.nan)
            flags.append(1)
            print(f'Warning: failed to load file {file}')
            continue

    #     print(data['block']['expDef'].split('\\')[-1])
    #     if 'contrastLeftValues' not in data['block']['events']:
        if 'expDef' in data['block']:
            exptype = data['block']['expDef'].split('\\')[-1]
        else:
            exptype = data['block']['expType']
        if 'blockWorld' not in exptype:
            ntrials.append(np.nan)
            nblocks.append(np.nan)
            print(f'Warning: file {file} is of type {exptype}, skipping...')
            flags.append(2)
            continue

        contrasts = data['block']['events']['contrastLeftValues']
        if type(contrasts) is int:
            contrasts = [contrasts]

        ntrials.append(len(contrasts))
        nblocks.append(np.sum(np.diff(contrasts) != 0))
        flags.append(0)
    
    return ntrials, nblocks, flags

In [5]:
# Let's parse the files to find out trial info etc
rigboxdir = '/Users/minhnhatle/Dropbox (MIT)/Nhat/Rigbox'
files = matlablog.value
ntrials, nblocks, flags = parse_files_for_trial_info(files)

  0%|          | 0/2203 [00:00<?, ?it/s]

In [6]:
# Clean, merge tables, and save
matlablog['ntrials'] = ntrials
matlablog['nblocks'] = nblocks
matlablog['flags'] = flags

matlablog['formatted_date'] = pd.DatetimeIndex(matlablog.dateStr).strftime('%Y-%m-%d')
googledb['formatted_date'] = pd.DatetimeIndex(googledb.date).strftime('%Y-%m-%d')


# Try to merge the google and matlab tables
mergedtbl = pd.merge(matlablog, googledb, on=['formatted_date', 'animal'])

file_to_store = open("explogs.pkl", "wb")
pickle.dump({'googledb': googledb, 'matlabdb': matlablog}, file_to_store)